In [2]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns

from DataFormat  import DataFormat
from read_data   import read_data
from calibration import read_calibration_parameters, linear_calibration

from matplotlib.colors import LogNorm

from tqdm.notebook import tqdm

## Constants

In [3]:
DATA_PATH   = "./data"
DATA_PREFIX = "beam-analysis-day"

LABR_A  = "labr-a"
LABR_B  = "labr-b"
CLYC    = "clyc"
PLASTIC = "plastic"

## Read data

In [17]:
run_days  = [2, 3]

data_labr_a = pd.DataFrame(
    columns=['board', 'channel', 'flag', 'timestamp', 'energy', 'energy_short', 'energy_calib', 'energy_short_calib', 'psd', 'psd_calib', 'id', 'Particle', 'run_day']
)
data_labr_b = pd.DataFrame(
    columns=['board', 'channel', 'flag', 'timestamp', 'energy', 'energy_short', 'energy_calib', 'energy_short_calib', 'psd', 'psd_calib', 'id', 'Particle', 'run_day']
)

dfs_a = []
dfs_b = []

for run_day in run_days:
    labr_a = read_data(DATA_PATH, f"{DATA_PREFIX}{run_day}.root", DataFormat(LABR_A))
    labr_a.loc[:, 'run_day'] = run_day
    labr_b = read_data(DATA_PATH, f"{DATA_PREFIX}{run_day}.root", DataFormat(LABR_B))
    labr_b.loc[:, 'run_day'] = run_day
    
    dfs_a.append(labr_a)
    dfs_b.append(labr_b)
    
data_labr_a = pd.concat(dfs_a)
data_labr_b = pd.concat(dfs_b)

In [19]:
# clean data from saturated events
data_labr_a = data_labr_a[data_labr_a["energy"] < data_labr_a["energy"].max()]
data_labr_b = data_labr_b[data_labr_b["energy"] < data_labr_b["energy"].max()]

# clean data from zero energy events
data_labr_a = data_labr_a[data_labr_a["energy"] > 0]
data_labr_b = data_labr_b[data_labr_b["energy"] > 0]

## Calibration

In [20]:
param_labr_a = read_calibration_parameters(
    "./calibration-parameters/labr_a_linear_parameters.txt"
)

param_labr_b = read_calibration_parameters(
    "./calibration-parameters/labr_b_linear_parameters.txt"
)

In [21]:
data_labr_a.loc[:, "energy_calib"]       = linear_calibration(data_labr_a["energy"], param_labr_a)
data_labr_a.loc[:, "energy_short_calib"] = linear_calibration(data_labr_a["energy_short"], param_labr_a)

data_labr_b.loc[:, "energy_calib"]       = linear_calibration(data_labr_b["energy"], param_labr_b)
data_labr_b.loc[:, "energy_short_calib"] = linear_calibration(data_labr_b["energy_short"], param_labr_b)

In [22]:
def compute_psd(df):
    return (df["energy"] - df["energy_short"]) / df["energy"]

def compute_psd_calib(df):
    return (df["energy_calib"] - df["energy_short_calib"]) / df["energy_calib"]

data_labr_a.loc[:, "psd"] = compute_psd(data_labr_a)
data_labr_b.loc[:, "psd"] = compute_psd(data_labr_b)
data_labr_a.loc[:, "psd_calib"] = compute_psd_calib(data_labr_a)
data_labr_b.loc[:, "psd_calib"] = compute_psd_calib(data_labr_b)

In [23]:
# remove all points that have a psd or psd_calib value greater than 1
def remove_psd_outliers(df):
    return df[(df["psd"] < 1) & (df["psd_calib"] < 1)]

data_labr_a = remove_psd_outliers(data_labr_a)
data_labr_b = remove_psd_outliers(data_labr_b)

In [24]:
# add an ID column to the dataframes that ranges from 0 to the number of events
data_labr_a.loc[:, "id"] = np.arange(len(data_labr_a))
data_labr_b.loc[:, "id"] = np.arange(len(data_labr_b))

In [25]:
# add a Particle column to the labr data with the value "Gamma" for all events
data_labr_a.loc[:, "Particle"] = "Gamma"
data_labr_b.loc[:, "Particle"] = "Gamma"

In [35]:
# consider energy calibrations between 480 and 540 keV
data_labr_a_511 = data_labr_a[(data_labr_a["energy_calib"] > 480) & (data_labr_a["energy_calib"] < 540)]
data_labr_b_511 = data_labr_b[(data_labr_b["energy_calib"] > 480) & (data_labr_b["energy_calib"] < 540)]

In [36]:
### LABR A ###
def check_timestamps_a(row, tw):
    global result_a
    
    # Get the timestamp of the current row
    timestamp = row['timestamp']

    # Find rows in the data_labr_a dataframe with timestamps within 100 nanoseconds of the current row's timestamp
    mask = (data_labr_a['timestamp'] >= timestamp - tw/2) & (data_labr_a['timestamp'] <= timestamp + tw/2)
    matching_rows = data_labr_a[mask]

    # If there are any matching rows, add them and the current row to the result dataframe
    if len(matching_rows) > 0:
        result_a = result_a.append(row, ignore_index=True)
        result_a = result_a.append(matching_rows, ignore_index=True)
        
        
### LABR B ###
def check_timestamps_b(row, tw):
    global result_b
    
    # Get the timestamp of the current row
    timestamp = row['timestamp']

    # Find rows in the data_labr_a dataframe with timestamps within 100 nanoseconds of the current row's timestamp
    mask = (data_labr_b['timestamp'] >= timestamp - tw/2) & (data_labr_b['timestamp'] <= timestamp + tw/2)
    matching_rows = data_labr_b[mask]

    # If there are any matching rows, add them and the current row to the result dataframe
    if len(matching_rows) > 0:
        result_b = result_b.append(row, ignore_index=True)
        result_b = result_b.append(matching_rows, ignore_index=True)

In [33]:
tqdm.pandas()

In [32]:
# time windows (in picoseconds) to check for coincidences
tw = 5000 # 5 ns

In [37]:
result_a = pd.DataFrame(columns=['board', 'channel', 'flag', 'timestamp', 'energy', 'energy_short', 'energy_calib', 'energy_short_calib', 'psd', 'psd_calib', 'id', 'Particle', 'run_day'])
data_labr_b_511.progress_apply(check_timestamps_a, axis=1, args=(tw,))

result_a.to_csv(f"{DATA_PATH}/labra-vs-labrb-coincidence-5ns.csv", index=False)

  0%|          | 0/1509774 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(f"Found {len(result_a)} coincidences (LaBrA) in {len(data_labr_b_511)} events (LaBrB) with energy between 480 and 540 keV.")

In [ ]:
result_b = pd.DataFrame(columns=['board', 'channel', 'flag', 'timestamp', 'energy', 'energy_short', 'energy_calib', 'energy_short_calib', 'psd', 'psd_calib', 'id', 'Particle', 'run_day'])
data_labr_a_511.progress_apply(check_timestamps_b, axis=1, args=(tw,))

result_b.to_csv(f"{DATA_PATH}/labrb-vs-labra-coincidence-5ns.csv", index=False)

In [ ]:
print(f"Found {len(result_b)} coincidences (LaBrB) in {len(data_labr_a_511)} events (LaBrA) with energy between 480 and 540 keV.")